## SISTEMA PARA IDENTIFICAÇÃO DE NOTÍCIAS FALSAS EM IDIOMA PORTUGUÊS POR MEIO DA UTILIZAÇÃO DE TÉCNICAS DE TEXT MINING 

## Importação das bibliotecas utilizadas

In [ ]:
import os
import pandas as pd 
import numpy as np 
import re 
import spacy 
from tqdm import tqdm 
import random

from spellchecker import SpellChecker
import unicodedata
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords 
from nltk.stem import SnowballStemmer 
from nltk.corpus import wordnet 

import matplotlib.pyplot as plt 
from matplotlib.ticker import PercentFormatter 

from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score 
from sklearn.svm import SVC 
from xgboost import XGBClassifier 
from sklearn.model_selection import train_test_split 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Dropout 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split

spell = SpellChecker(language='pt') 
nltk.download('stopwords') 
nltk.download('omw-1.4')

## Funções dataprep

In [ ]:
def conta_palavra(texto):
    word_count = {}
    words = texto.split() 
    for word in words: 
        word = word.lower() 
        if not word in word_count: 
            word_count[word] = 1 
        else: 
            word_count[word] = word_count[word] + 1          

    df_columns = pd.DataFrame([word_count])
    df_stat = pd.DataFrame(word_count.items(), columns=['palavra', 'qtd']) 

    return(df_stat, df_columns) 

In [ ]:
def corrigir_ortografia(texto): 
    palavras = texto.split() 
    texto_corrigido = ''
    for i, palavra in enumerate(palavras): 
        try:
            correcao = spell.correction(palavra)
        except: 
            correcao = palavra 

        if correcao == None:
            correcao = palavra
        
        if i == 0:
            texto_corrigido = correcao
        else:
            texto_corrigido = texto_corrigido + ' ' + correcao 

    return texto_corrigido 

In [ ]:
def reconhecer_entidades_nomeadas(texto):
    nlp = spacy.load("pt_core_news_sm")
    doc = nlp(texto)
    entidades = [(entidade.text, entidade.label_) for entidade in doc.ents]

    for entidade in entidades:
                    texto = texto.replace(entidade[0], 'entidade_' + entidade[1]) 

    return texto 

In [ ]:
def remover_stopwords(texto):
    stopwords_pt = set(stopwords.words('portuguese')) 
    texto_sem_stopwords = ''
    texto = unidecode(texto) 
    palavras = texto.split() 

    for i, palavra in enumerate(palavras): 
        if palavra.lower() not in stopwords_pt:
            if texto_sem_stopwords == '':
                texto_sem_stopwords = palavra.lower()
            else:
                texto_sem_stopwords = texto_sem_stopwords + ' ' + palavra.lower() 
    return texto_sem_stopwords 

In [ ]:
def realizar_stemming(texto): 
    stemmer = SnowballStemmer("portuguese")
    palavras = texto.split()

    texto_stemmed = '' 
    for i, palavra in enumerate(palavras):
        palavra_stemming = stemmer.stem(palavra)
        if i == 0:
            texto_stemmed = palavra_stemming
        else:
            texto_stemmed = texto_stemmed + ' ' + palavra_stemming
        
        dict_stemming[palavra_stemming] = palavra 

    return texto_stemmed 


In [ ]:
def obter_sinonimos(palavra):
    sinonimos = set()
    for syn in wordnet.synsets(palavra, lang="por"):
        for lemma in syn.lemmas(lang="por"):
            sinonimos.add(lemma.name())
    
    return list(sinonimos)

In [ ]:
def manipulacao_sinonimos(texto, dicionario_de_sinonimos):
    palavras = nltk.word_tokenize(texto)
    novo_texto = []   

    for palavra in palavras:
        if palavra in dicionario_de_sinonimos: 
            sinonimos = dicionario_de_sinonimos[palavra] 
            if sinonimos: 
                novo_texto.append(sinonimos[0])# Substitui pela primeira sugestão de sinônimo
            
            else:
                novo_texto.append(palavra) 
        else:
            novo_texto.append(palavra) 

    return ' '.join(novo_texto) 

In [ ]:
def bag_of_words(textos, palavras):
    word_counts = {word: [] for word in palavras}

    for texto in textos:
        counts = {word: 0 for word in palavras} 
        text_words = texto.split() 
        for word in text_words: 
            if word in counts: 
                counts[word] += 1 
        for word in palavras: 
            word_counts[word].append(counts[word])
			
    df = pd.DataFrame(word_counts)     

    return df 

## Leitura dos Dados

In [ ]:
pasta = r'data\full_texts' 
lista_texto_fake = []
lista_texto_true = []
	
for diretorio, subpastas, arquivos in os.walk(pasta): 
    for subpasta in subpastas:
        if subpasta in ['fake','true']: 
            for arquivo in (os.listdir(pasta + '/' + subpasta)): 
                with open(pasta + '/' + subpasta + '/' + arquivo, encoding="utf8") as f: 
                    texto_atual = f.read() 
                    if subpasta == 'fake': 
                        lista_texto_fake.append(texto_atual) 
                    else:
                        lista_texto_true.append(texto_atual) 

In [ ]:
amostra_fake = random.sample(lista_texto_fake, int(len(lista_texto_fake) * 0.10)) 
amostra_true = random.sample(lista_texto_true, int(len(lista_texto_true) * 0.10))

In [ ]:
lista_texto_amostral = amostra_fake + amostra_true
lista_texto_amostral_trat = [] 

for texto in tqdm(lista_texto_amostral): 
    texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto) 
    texto_corrigido = corrigir_ortografia(texto) 
    texto_entidate = reconhecer_entidades_nomeadas(texto_corrigido) 

    lista_texto_amostral_trat.append(texto_entidate)

In [ ]:
texto_pre_tratado = '' 
for texto in lista_texto_amostral_trat:
    texto_pre_tratado = texto_pre_tratado + " " + texto  

## Definição da quantidade de entradas e palavras selecionadas 

In [ ]:
df_stat, df_columns = conta_palavra(texto_pre_tratado) 
df_stat 

dict_stemming = {} 

df = df_stat.copy() 
df.index = df['palavra'].values 
df = df.sort_values(by='qtd',ascending=False) 
df["porcentagem_acumulada"] = df["qtd"].cumsum()/df["qtd"].sum()*100 
df['palavra_original'] = df['palavra'].map(dict_stemming)

In [ ]:
fig, ax = plt.subplots() 
ax.bar(df.index, df["qtd"], color="C0") 
ax2 = ax.twinx() 
ax2.plot(df.index, df["porcentagem_acumulada"], color="C1", marker="D", ms=3) 
ax2.yaxis.set_major_formatter(PercentFormatter()) 

ax.tick_params(axis="y", colors="C0") 
ax2.tick_params(axis="y", colors="C1") 
plt.show()

In [ ]:
df_filtrado = df[((df["porcentagem_acumulada"]<=65) & (df["palavra"].str.len() >= 3))] 
print(len(df_filtrado['palavra'].values)) 

In [ ]:
dicionario_de_sinonimos = {}
    
for palavra_original in df_filtrado['palavra'].values: 
    dicionario_de_sinonimos[palavra_original] = obter_sinonimos(palavra_original) 


## Pré processamento total 

In [ ]:
lista_texto_fake_trat_total = []
lista_texto_true_trat_total = []

with open('fake_trat.txt', 'w') as f:
    for texto in tqdm(lista_texto_fake): 
        texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto)
        texto_corrigido = corrigir_ortografia(texto)
        texto_entidate = reconhecer_entidades_nomeadas(texto_corrigido)
        texto_com_manipulacao_sinonimos = manipulacao_sinonimos(texto_entidate, dicionario_de_sinonimos)
        texto_stop = remover_stopwords(texto_com_manipulacao_sinonimos)
        texto_stemming = realizar_stemming(texto_stop)
        
        f.write(texto_stemming)
        f.write('\n') 

with open('true_trat.txt', 'w') as f: 
    for texto in tqdm(lista_texto_true):
        texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto)
        texto_corrigido = corrigir_ortografia(texto)
        texto_entidate = reconhecer_entidades_nomeadas(texto_corrigido)
        texto_com_manipulacao_sinonimos = manipulacao_sinonimos(texto_entidate, dicionario_de_sinonimos)
        texto_stop = remover_stopwords(texto_com_manipulacao_sinonimos)
        texto_stemming = realizar_stemming(texto_stop)	

        f.write(texto_stemming)
        f.write('\n')

## Criação da tabela para modelagem 

In [ ]:
lista_texto_fake_trat_total = []
lista_texto_true_trat_total = []

with open('fake_trat.txt') as f:
    lista_texto_fake_trat_total = f.readlines()
	
with open('true_trat.txt') as f:
    lista_texto_true_trat_total = f.readlines()
	
lista_palavras_bagofwords = df_filtrado['palavra'].values 

df_fake = bag_of_words(lista_texto_fake_trat_total, lista_palavras_bagofwords)
df_fake['target'] = 1	

df_true = bag_of_words(lista_texto_true_trat_total, lista_palavras_bagofwords)
df_true['target'] = 0

df_total = df_fake.append(df_true).reset_index(drop=True)
df_total 

## Separação da tabela final em treino, teste e validação 

In [ ]:
train_df, rest_df = train_test_split(df_total, test_size=0.3, random_state=42, stratify=df_total["target"])
val_df, test_df = train_test_split(rest_df, test_size=0.5, random_state=42)

X_train, y_train = train_df.drop('target', axis=1), train_df['target']
X_val, y_val = val_df.drop('target', axis=1), val_df['target']
X_test, y_test = test_df.drop('target', axis=1), test_df['target']

scaler = StandardScaler()
X_train_2 = scaler.fit_transform(X_train)
X_val_2 = scaler.transform(X_val)
X_test_2 = scaler.transform(X_test)	

X_train_lstm = X_train_2.reshape(X_train_2.shape[0], 1, X_train_2.shape[1])
X_val_lstm = X_val_2.reshape(X_val_2.shape[0], 1, X_val_2.shape[1])
X_test_lstm = X_test_2.reshape(X_test_2.shape[0], 1, X_test_2.shape[1]) 

## Modelos de Aprendizado de Máquina

In [ ]:
models = {
    "XGBoost": {
        "model": XGBClassifier(),
        "params": {
            "n_estimators": [100, 200, 300],
            "max_depth": [3, 5, 7],
        },
    },
    "SVM": {
        "model": SVC(probability=True),
        "params": {
            "C": [0.1, 1, 10],
            "kernel": ["linear", "rbf"],
        },
    },
    "LogisticRegression": {
        "model": LogisticRegression(),
        "params": {
            "C": [0.1, 1, 10],
        },
    },
}	

## Modelos de Redes Neurais

In [ ]:
def create_dense_nn():
    model = Sequential()
    model.add(Dense(128, input_dim=311, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
	

def create_lstm_nn():
    model = Sequential()
    model.add(LSTM(128, input_shape=(1, 311), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model 

In [ ]:
models_nn = {
    "Dense_NN": create_dense_nn(),
    "LSTM": create_lstm_nn(),
}

results_df = pd.DataFrame(columns=["Model", "Model_Name", "Set", "Accuracy", "Precision", "Recall", "F1-Score", "ROC AUC"])

## Treinamento, validação e avaliação de métricas para modelos de aprendizado de máquina

In [ ]:
for model_name, model_info in models.items():
    model = model_info["model"]
    params = model_info["params"]
    
    # GridSearchCV
    grid_search = GridSearchCV(model, params, cv=3, verbose=1, n_jobs=-1, scoring='roc_auc')
    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_
    
    # Avalição do modelo no conjunto de treinamento
    y_train_pred = best_model.predict(X_train)
    y_train_prob = best_model.predict_proba(X_train)[:, 1]
    
    accuracy = accuracy_score(y_train, y_train_pred)
    precision = precision_score(y_train, y_train_pred)
    recall = recall_score(y_train, y_train_pred)
    f1 = f1_score(y_train, y_train_pred)
    roc_auc = roc_auc_score(y_train, y_train_prob)
    
    results_df = results_df.append({"Model": model, "Model_Name": model_name, "Set": "Train", "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1-Score": f1, "ROC AUC": roc_auc}, ignore_index=True)

    # Avalição do modelo no conjunto de validação
    y_val_pred = best_model.predict(X_val)
    y_val_prob = best_model.predict_proba(X_val)[:, 1]
    
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    f1 = f1_score(y_val, y_val_pred)
    roc_auc = roc_auc_score(y_val, y_val_prob)
    results_df = results_df.append({"Model": model, "Model_Name": model_name, "Set": "Validation", "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1-Score": f1, "ROC AUC": roc_auc}, ignore_index=True)     

    # Avalição do modelo no conjunto de teste
    y_test_pred = best_model.predict(X_test)
    y_test_prob = best_model.predict_proba(X_test)[:, 1]
    
    accuracy = accuracy_score(y_test, y_test_pred)
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)
    roc_auc = roc_auc_score(y_val, y_test_prob)
    
    results_df = results_df.append({"Model": model, "Model_Name": model_name, "Set": "Test", "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1-Score": f1, "ROC AUC": roc_auc}, ignore_index=True)
     


## Treinamento, validação e avaliação de métricas para modelos de redes neurais

In [ ]:
for model_name, model in models_nn.items():
    clf = model
    if model_name == 'LSTM':
        X_train = X_train_lstm 
        X_val = X_val_lstm
        X_test = X_test_lstm
        
    clf.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)
    
    # Avalição do modelo no conjunto de treinamento
    y_train_prob = clf.predict(X_train)
    
    accuracy = accuracy_score(y_train, y_train_pred)
    precision = precision_score(y_train, y_train_pred)
    recall = recall_score(y_train, y_train_pred)
    f1 = f1_score(y_train, y_train_pred)
    roc_auc = roc_auc_score(y_train, y_train_prob)
    
    results_df = results_df.append({"Model": clf, "Model_Name": model_name, "Set": "Train", "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1-Score": f1, "ROC AUC": roc_auc}, ignore_index=True)

    # Avalição do modelo no conjunto de validação
    y_val_prob = clf.predict(X_val)
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    f1 = f1_score(y_val, y_val_pred)
    roc_auc = roc_auc_score(y_val, y_val_prob)
    
    results_df = results_df.append({"Model": clf, "Model_Name": model_name, "Set": "Validation", "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1-Score": f1, "ROC AUC": roc_auc}, ignore_index=True) 
     
    # Avalição do modelo no conjunto de teste
    y_test_prob = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_test_pred)
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)
    roc_auc = roc_auc_score(y_test, y_test_prob)
    
    results_df = results_df.append({"Model": clf, "Model_Name": model_name, "Set": "Test", "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1-Score": f1, "ROC AUC": roc_auc}, ignore_index=True)